In [21]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from joblib import dump

In [10]:
d = pd.read_csv('toe.csv')
d.head()

,Отметка времени,Итоговая оценка за экзамен по ТОЭ,"С какого раза вы сдали экзамен по ТОЭ (если улучшали оценку, так же указать с какого раза получили желаемую оценку)","Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?",Оценка по физике в школе,Оценка по физике в последнем семестре?,Оценка за ИДЗ по ТОЭ за семестр,Оценка по лабораторным ТОЭ,Вели конспект по занятиям? (лекции и/или практики),Выходили к доске на практике?,...,В течении предыдущего семестра были трудоустроены и/или проходили стажировку?,"Если работали/проходили стажировку, то какая ставка (загрузка):",Где проживаете?,"Есть кот дома? (в месте, где сейчас живете)",Есть родной брат/сестра?,Гражданство,Сколько в среднем играли в прошлом семестре? Укажите число в часах,Была ли стипендия в 3 семестре?,Оцените свое ментальное здоровье в прошлом семестре,"Оцените, насколько были истощены (уставши) к началу сессии"
0,2024/09/30 11:51:10 PM GMT+3,5.0,1.0,75,5,5.0,5.0,5.0,да,да,...,нет,0.00,общежитие 8,нет,да,KAZ,10,да,9,1
1,2024/09/30 11:53:32 PM GMT+3,3.0,1.0,75,4,5.0,4.0,5.0,да,нет,...,нет,0.00,дома,нет,да,RUS,20,нет,7,6
2,2024/09/30 11:53:33 PM GMT+3,3.0,1.0,75,5,4.0,5.0,5.0,да,нет,...,да,0.25,дома,да,да,RUS,168,да,7,3
3,2024/09/30 11:57:50 PM GMT+3,3.0,1.0,75,5,4.0,4.0,4.0,нет,нет,...,да,0.50,общежитие 8,нет,да,KAZ,18,нет,7,2
4,2024/09/30 11:59:56 PM GMT+3,4.0,1.0,100,4,3.0,4.0,5.0,нет,нет,...,нет,NaN,дома,нет,да,RUS,20,нет,10,5


In [3]:
#признаки
f = ['Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?',
            'Оценка по физике в школе',
            'Оценка по физике в последнем семестре?', 
            'Оценка за ИДЗ по ТОЭ за семестр ',
            'Оценка по лабораторным ТОЭ',
            'Вели конспект по занятиям? (лекции и/или практики)', 
            'Выходили к доске на практике?',
            'В течении предыдущего семестра были трудоустроены и/или проходили стажировку?',
            'Если работали/проходили стажировку, то какая ставка (загрузка):', 
            'Есть кот дома? (в месте, где сейчас живете)',
            'Есть родной брат/сестра?',
            'Сколько в среднем играли в прошлом семестре? Укажите число в часах',
            'Была ли стипендия в 3 семестре?',
            'Оцените свое ментальное здоровье в прошлом семестре',
            'Оцените, насколько были истощены (уставши) к началу сессии'
           ] 


In [5]:
# Преобразование категориальных признаков в числовые
d['Вели конспект по занятиям? (лекции и/или практики)'] = d['Вели конспект по занятиям? (лекции и/или практики)'].map({'да': 1, 'нет': 0})
d['Выходили к доске на практике?'] = d['Выходили к доске на практике?'].map({'да': 1, 'нет': 0})
d['В течении предыдущего семестра были трудоустроены и/или проходили стажировку?'] = d['В течении предыдущего семестра были трудоустроены и/или проходили стажировку?'].map({'да': 1, 'нет': 0, '...': 0.5}) # Заменяем "..." на среднее значение
d['Есть кот дома? (в месте, где сейчас живете)'] = d['Есть кот дома? (в месте, где сейчас живете)'].map({'да': 1, 'нет': 0})
d['Есть родной брат/сестра?'] = d['Есть родной брат/сестра?'].map({'да': 1, 'нет': 0})
d['Была ли стипендия в 3 семестре?'] = d['Была ли стипендия в 3 семестре?'].map({'да': 1, 'нет': 0})
d['Сколько в среднем играли в прошлом семестре? Укажите число в часах'] = pd.to_numeric(d['Сколько в среднем играли в прошлом семестре? Укажите число в часах'], errors='coerce').fillna(0)

X = d[f]
y = d['Итоговая оценка за экзамен по ТОЭ']

In [6]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
m = LinearRegression()

In [13]:
# Обучение модели
d.dropna(axis=1, how='all', inplace=True)
X_train = X_train[y_train.notna()]
y_train = y_train.dropna()

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

m.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = m.predict(X_test)

In [19]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [23]:
print(f"Средняя абсолютная ошибка (MAE): {mae}")
print(f"Коэффициент детерминации (R^2): {r2}")

# Сохранение модели
filename = 'model.pkl'
pickle.dump(m, open(filename, 'wb'))

Средняя абсолютная ошибка (MAE): 0.7344610180237225
Коэффициент детерминации (R^2): -0.49043640732387805
